In [ ]:
import os
import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

Data was organised into folders such that the parameter being optimised in each dataset is the name of the parent folder

In [ ]:
folders = glob.glob('*ECSTM*') # Set folder to search current directory

# Choose optimising parameter
# parameter = 'bias'
# parameter = 'setpoint'

# List all valid paths
paths = [os.path.join(os.getcwd(), x, parameter) for x in folders]
paths

In [ ]:
def find_start_line(lines):
    '''
    Returns the line index following the ASCII heading.
    '''
    for i, line in enumerate(lines):
        if 'ASCII' in line:
            return i + 1
    return -1

def format_data(data):
    '''
    Converts read raw lines of text data into a numpy array.
    '''
    Converts read raw lines of text data into a numpy array.
    currents = []
    for line in data:
        line = line.split('\n')[0]
        line = line.split('\t')
        current = float(line[-1])
        currents.append(current)
    return np.array(currents)


def load_data_from_lines(lines):
    '''
    importdata helper function.
    '''
    data_start_line = find_start_line(lines) # Finds the line where data starts including column headers
    header = lines[data_start_line] # Extracts the column headers
    data = lines[data_start_line+1:] # Raw data lines
    data = format_data(data) # Formats the raw data lines
    return data

def importdata(file):
    '''
    Loads data from a PicoScan I(t) txt file. Returns both the data and text header.
    '''
    f = open(file, 'r')
    lines = f.readlines()
    f.close()
    bias = float(file.split('/')[-1].split('.txt')[0])
    data = load_data_from_lines(lines)
    return bias, data

In [ ]:
def analyse_path(path):
    '''
    Imports I(t) data from path. Imported data is then formated into a DataFrame.
    '''
    files = glob.glob(os.path.join(path, '*.txt'))
    
    vals = [[], []]
    data_dict = dict(zip(data_cols[1:], vals))
    
    for j, file in enumerate(files):
        print(file)
        val, data = importdata(file)
        data_dict[parameter].append(val)
        data_dict['data'].append(data)
    df = pd.DataFrame(data_dict)
    return df

# Construct and populate DataFrame to contain all experiment's data
data_cols = ['run', parameter, 'data']
data_df = pd.DataFrame(columns=data_cols)
for i, path in enumerate(paths):
    print(path)
    df = analyse_path(path)
    df['run'] = i
    data_df = pd.concat((data_df, df))

data_df

## Calculate Trace Noise

In [ ]:
# Calculates a new DataFrame containing the noise of each run
cols = ['run', parameter, 'std']
vals = [[], [], []]
std_dict = dict(zip(cols, vals))

for idx, row in data_df.iterrows():
    run = row.run
    par = row[parameter]
    std = row.data.std()
    
    std_dict['run'].append(run)
    std_dict[parameter].append(par)
    std_dict['std'].append(std)
    
RMS_df = pd.DataFrame(std_dict)
RMS_df

## Visualise Noise Results

In [ ]:
# Plot noise vs parameter value for each experiment
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)

sns.lineplot(data=RMS_df, x=parameter, y='std', hue='run', ax=ax)

ax.set_ylabel('Standard Deviation / nA', weight='bold')
ax.set_xlabel(parameter.title() + ' / V', weight='bold')
ax.get_legend().remove()
ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)

fig.savefig("BiasNoise.png")

In [ ]:
# Display the average noise for each parameter value
RMS_df.groupby([parameter]).agg('mean')

In [ ]:
# Display the standard deviation of noise RMS for each parameter value
RMS_df.groupby([parameter]).agg('std')